![title](wiot.jpg)

![title](REII.jpg)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
wiot2014 = pd.read_csv('wiot14.csv', low_memory = False)

In [3]:
print(wiot2014.shape)
wiot2014.head()

(2473, 2687)


,Unnamed: 0,Unnamed: 1,AUS,AUS.1,AUS.2,AUS.3,AUS.4,AUS.5,AUS.6,AUS.7,...,USA.57,USA.58,USA.59,USA.60,ROW.56,ROW.57,ROW.58,ROW.59,ROW.60,TOT
0,NaN,NaN,c1,c2,c3,c4,c5,c6,c7,c8,...,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!,#REF!,c62
1,AUS,r1,12924.17969,112.4542209,228.480462,538.8908067,25139.33429,737.4925912,4.475429881,3.130540974,...,0,3.56539E-05,0.324914354,0.027391046,723.1899841,0.000374341,14.15634984,20.52070767,0.010083053,70292.03449
2,AUS,r2,83.02963718,201.4999483,0.191312931,17.19067922,2.646330705,0.09376783,1430.804904,0.36945428,...,0,1.64768E-06,0.015015318,0.000438507,0.423421659,2.92886E-06,7.05418E-05,0.000976644,-0.000109455,2585.379685
3,AUS,r3,19.14773092,0.001078923,19.10670051,4.228107625,316.075149,22.86668291,0.252293929,0.268537087,...,0,5.44583E-07,0.004962784,0.008311129,404.0411387,0.000386117,0.004618955,0.357977855,0.000591879,3175.044396
4,AUS,r4,115.929847,0.695039643,5.924476501,4492.037652,331.6093559,5.984140757,29.74092405,49.7639612,...,0,0.000176703,1.610294116,0.047026998,14.87162738,0.006664525,0.594704542,19.7268333,-0.282072196,171985.1223


In [4]:
# main elements of WIOT
Z = wiot2014.iloc[1:2465, 2:2466] # intermediate use
Z = Z.astype(np.float)
Z.index = Z.columns

VALU = wiot2014.iloc[2471, 2:2466] # value added
VALU = VALU.astype(np.float)

X = wiot2014.iloc[2472, 2:2466] # output
X = X.astype(np.float)

FD = wiot2014.iloc[2:2465, 2466:2685] # final demand
FD = FD.astype(np.float)

In [5]:
print(Z.shape)
print(VALU.shape)
print(X.shape)
print(FD.shape)

(2464, 2464)
(2464,)
(2464,)
(2463, 219)


In [6]:
# placeholders
# empty_inner - ones, where c = p
empty_inner = np.empty(shape = Z.shape)
prev = 0
next_ = 0
for n in range(0, 2463, 44):
    prev = n
    next_ = prev + 44
    empty_inner[prev:next_, prev:next_] = 1
# empty_outer - ones, where c ≠ p
empty_outer = np.ones(shape = Z.shape)
prev = 0
next_ = 0
for n in range(0, 2463, 44):
    prev = n
    next_ = prev + 44
    empty_outer[prev:next_, prev:next_] = 0

In [7]:
empty_inner

array([[1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        2.32885185e-314, 2.32885188e-314, 2.24007505e-314],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        2.35223129e-314, 2.35223135e-314, 2.24007505e-314],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        2.35230310e-314, 2.35230313e-314, 2.24007505e-314],
       ...,
       [2.72859950e-314, 2.72859953e-314, 2.72859956e-314, ...,
        1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       [2.24007505e-314, 2.24007505e-314, 2.24007505e-314, ...,
        1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       [2.72867184e-314, 2.72867187e-314, 2.72867191e-314, ...,
        1.00000000e+000, 1.00000000e+000, 1.00000000e+000]])

In [8]:
empty_outer

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [9]:
# PROD - Gross output by industry and country
PROD = Z.sum(axis = 1)
print(PROD.shape)

(2464,)


In [10]:
# A - Input coefficients
A = Z / PROD
print(A.shape)

(2464, 2464)


In [11]:
A.replace(np.inf, 0, inplace = True) # division by zero errors
A.replace(np.nan, 0, inplace = True) # division by zero errors

In [12]:
# B - Global Leontief inverse
B = np.linalg.inv(np.eye(Z.shape[0]) - A)
print(B.shape)

(2464, 2464)


In [13]:
# EXGR - represents a K x 1 vector of gross exports from country c to country p for all K industries, where c ≠ p.
EXGR = (Z * empty_outer).sum(axis = 1)
print(EXGR.shape)

(2464,)


In [14]:
# REII - re-exported intermediate imports
REII = np.dot(np.dot(A, B), EXGR)

In [15]:
print('REII min = ', REII.min(), 'mio USD')
print('REII max = ', REII.max(), 'mio USD')
print('REII mean = ', REII.mean(), 'mio USD')
print('REII median = ', np.median(REII), 'mio USD')

REII min =  0.0 mio USD
REII max =  2214165653.077099 mio USD
REII mean =  30219180.827729 mio USD
REII median =  3949721.337452318 mio USD


In [16]:
print(REII.shape)

(2464,)
